In [1]:
import cv2
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout 
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from matplotlib import pyplot as plt
import splitfolders
import pandas as pd
from keras.utils import np_utils
import numpy as np
import tensorflow as tf

In [ ]:
inputPath='E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Input'
outputPath='E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output'
splitfolders.ratio(inputPath, output=outputPath, seed=1337, ratio=(0.8,0.0,0.2))

In [10]:
X_train = []
Y_train = []
X_train = np.asarray(X_train)

trainPath = 'E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output\\train'
for folder in os.listdir(trainPath):
    for image in os.listdir(os.path.join(trainPath, folder)):
        X_train.append(cv2.imread(os.path.join(trainPath, os.path.join(folder,image))))
        Y_train.append(folder)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [32]:
print(len(X_train))
print(len(Y_train))

21600
21600


In [7]:
X_test = []
Y_test = []
testPath = 'E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output\\test'
for folder in os.listdir(testPath):
    for image in os.listdir(os.path.join(testPath, folder)):
        X_test.append(cv2.imread(os.path.join(testPath, os.path.join(folder,image))))
        Y_test.append(folder)

In [8]:
print(len(X_test))
print(len(Y_test))

5400
5400


In [38]:
print(X_train.shape)
print(X_test.shape)

(21600,)
(5400, 64, 64, 3)


In [37]:
X_train = np.array(X_train,dtype=object)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [13]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')

ValueError: setting an array element with a sequence.

In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(64,64,3), padding='same', activation='relu'))
model.add(Dropout(0.15))
model.add(Conv2D(32, (3,3), input_shape=(64,64,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=2,padding='valid'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4194432   
                                                        

In [10]:
import random 
buf = list(zip(X_train,Y_train))
random.shuffle(buf)
X_train,Y_train = zip(*buf)
buf = list(zip(X_test,Y_test))
random.shuffle(buf)
X_test,Y_test = zip(*buf)


In [11]:
X_train = np.array(X_train)
Y_train=np.array(Y_train)
X_test=np.array(X_test)
Y_test=np.array(Y_test)

In [12]:
results = {
    'AnnualCrop':0,
    'Forest':1,
    'HerbaceousVegetation':2,
    'Highway':3,
    'Industrial':4,
    'Pasture':5,
    'PermanentCrop':6,
    'Residential':7,
    'River':8,
    'SeaLake':9
}
y_train = []
for i in Y_train:
    y_train.append(results.get(i))

In [14]:
y_train = np_utils.to_categorical(y_train)
print(y_train[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [15]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(21600, 64, 64, 3)
(21600,)
(5400, 64, 64, 3)
(5400,)


In [16]:
scale = X_train.max()
X_train = X_train/scale
X_test = X_test/scale

mean = X_train.mean()
X_train = X_train - mean
X_test = X_test - mean

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1)

Epoch 1/10
